In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from skimage.io import imread_collection

In [8]:
# load image examples into a collection
examples = imread_collection("i/*.png")
print(examples)

['i\\baboon.png', 'i\\butterfly.png', 'i\\city.png', 'i\\house.png', 'i\\seagull.png']
